# 1. Generación del constrain "mínima cantidad de docentes requeridos"

## Importacion de librerías

In [1]:
import pandas as pd

## Extraemos la demanda estudiantil

In [2]:
enlace_matriculados="..\\Data\\Origen\\matriculados2025I.csv"

In [3]:
df2=pd.read_csv(enlace_matriculados,sep=';')
df2.head()

,ciclo,Asignatura,Créd.,Sec.,Docente,Tope,Matriculados
0,ciclo I,INE002 - PROGRAMACIÓN Y COMPUTACIÓN,2,2,"0A0182 - RUIZ RIVERA, MARIA ELENA",4,2
1,ciclo I,INE002 - PROGRAMACIÓN Y COMPUTACIÓN,2,3,"07240690 - DAMASO RIOS, MARIA ROSA",4,3
2,ciclo I,INE002 - PROGRAMACIÓN Y COMPUTACIÓN,2,5,"085774 - VERA POMALAZA, VIRGINIA",4,1
3,ciclo I,INE002 - PROGRAMACIÓN Y COMPUTACIÓN,2,6,"10645375 - USCUCHAGUA FLORES, GELBER CHRISTIAN",4,3
4,ciclo I,INO101 - REDACCIÓN Y TÉCNICAS DE COMUNICACIÓN ...,3,4,"15584946 - DIONICIO MEJÍA, CARMEN VIOLETA",4,2


In [4]:
df2[['CodigoAsignatura', 'NombreAsignatura']] = df2['Asignatura'].str.split(' - ', n=1, expand=True)

In [5]:
df2.head()

,ciclo,Asignatura,Créd.,Sec.,Docente,Tope,Matriculados,CodigoAsignatura,NombreAsignatura
0,ciclo I,INE002 - PROGRAMACIÓN Y COMPUTACIÓN,2,2,"0A0182 - RUIZ RIVERA, MARIA ELENA",4,2,INE002,PROGRAMACIÓN Y COMPUTACIÓN
1,ciclo I,INE002 - PROGRAMACIÓN Y COMPUTACIÓN,2,3,"07240690 - DAMASO RIOS, MARIA ROSA",4,3,INE002,PROGRAMACIÓN Y COMPUTACIÓN
2,ciclo I,INE002 - PROGRAMACIÓN Y COMPUTACIÓN,2,5,"085774 - VERA POMALAZA, VIRGINIA",4,1,INE002,PROGRAMACIÓN Y COMPUTACIÓN
3,ciclo I,INE002 - PROGRAMACIÓN Y COMPUTACIÓN,2,6,"10645375 - USCUCHAGUA FLORES, GELBER CHRISTIAN",4,3,INE002,PROGRAMACIÓN Y COMPUTACIÓN
4,ciclo I,INO101 - REDACCIÓN Y TÉCNICAS DE COMUNICACIÓN ...,3,4,"15584946 - DIONICIO MEJÍA, CARMEN VIOLETA",4,2,INO101,REDACCIÓN Y TÉCNICAS DE COMUNICACIÓN EFECTIVA I


In [6]:
cantidad_matriculados=df2.groupby(["CodigoAsignatura","NombreAsignatura"])["Matriculados"].sum().reset_index()
cantidad_matriculados

,CodigoAsignatura,NombreAsignatura,Matriculados
0,202W0301,ALGORÍTMICA I,10
1,202W0302,ESTADÍSTICA,5
2,202W0303,FISICA ELÉCTRONICA,6
3,202W0304,INGENIERÍA ECONÓMICA,2
4,202W0305,INTRODUCCIÓN AL DESARROLLO DE SOFTWARE,4
...,...,...,...
58,INO102,MÉTODOS DE ESTUDIO UNIVERSITARIO,1
59,INO103,DESARROLLO PERSONAL Y LIDERAZGO,1
60,INO106,ÁLGEBRA Y GEOMETRÍA ANALÍTICA,3
61,INO204,CÁLCULO II,2


## Obtenemos la demanda y la duración por curso

In [7]:
duracion_asignatura=pd.read_csv("..\\Data\\duracion_asignatura.csv",sep=",",encoding='utf-8-sig')
duracion_asignatura.head()

,CodigoAsignatura,NombreAsignatura,DuracionHoras
0,202W0301,ALGORÍTMICA I,5.0
1,202W0301,ALGORÍTMICA I,3.0
2,202W0302,ESTADÍSTICA,5.0
3,202W0303,FISICA ELÉCTRONICA,4.0
4,202W0304,INGENIERÍA ECONÓMICA,4.0


In [8]:
df3=cantidad_matriculados
df3=df3.merge(duracion_asignatura[["NombreAsignatura","DuracionHoras"]],how="left",on="NombreAsignatura")
df3

,CodigoAsignatura,NombreAsignatura,Matriculados,DuracionHoras
0,202W0301,ALGORÍTMICA I,10,5.0
1,202W0301,ALGORÍTMICA I,10,3.0
2,202W0302,ESTADÍSTICA,5,5.0
3,202W0303,FISICA ELÉCTRONICA,6,4.0
4,202W0304,INGENIERÍA ECONÓMICA,2,4.0
...,...,...,...,...
60,INO102,MÉTODOS DE ESTUDIO UNIVERSITARIO,1,3.0
61,INO103,DESARROLLO PERSONAL Y LIDERAZGO,1,3.0
62,INO106,ÁLGEBRA Y GEOMETRÍA ANALÍTICA,3,6.0
63,INO204,CÁLCULO II,2,6.0


In [9]:
# Parámetros
max_estudiantes_por_grupo = 60
max_horas_por_docente = 12

In [10]:
import math

In [11]:
# Cálculo de grupos y horas totales requeridas
df3['NumGrupos'] = df3['Matriculados'].apply(lambda d: math.ceil(d / max_estudiantes_por_grupo))
df3['HorasTotales'] = df3['NumGrupos'] * df3['DuracionHoras']

# Suma total de horas a cubrir
total_horas_a_cubrir = df3['HorasTotales'].sum()

# Cálculo de docentes mínimos
docentes_minimos = math.ceil(total_horas_a_cubrir / max_horas_por_docente)

df3


,CodigoAsignatura,NombreAsignatura,Matriculados,DuracionHoras,NumGrupos,HorasTotales
0,202W0301,ALGORÍTMICA I,10,5.0,1,5.0
1,202W0301,ALGORÍTMICA I,10,3.0,1,3.0
2,202W0302,ESTADÍSTICA,5,5.0,1,5.0
3,202W0303,FISICA ELÉCTRONICA,6,4.0,1,4.0
4,202W0304,INGENIERÍA ECONÓMICA,2,4.0,1,4.0
...,...,...,...,...,...,...
60,INO102,MÉTODOS DE ESTUDIO UNIVERSITARIO,1,3.0,1,3.0
61,INO103,DESARROLLO PERSONAL Y LIDERAZGO,1,3.0,1,3.0
62,INO106,ÁLGEBRA Y GEOMETRÍA ANALÍTICA,3,6.0,1,6.0
63,INO204,CÁLCULO II,2,6.0,1,6.0


In [12]:
print(f"\nTotal de horas a cubrir: {total_horas_a_cubrir}")
print(f"Docentes mínimos estimados: {docentes_minimos}")


Total de horas a cubrir: 329.0
Docentes mínimos estimados: 28
